<a href="https://colab.research.google.com/github/drpetros11111/LazyProgrammer/blob/TF-IDF/LP_RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# https://www.kaggle.com/tmdb/tmdb-movie-metadata
!wget https://lazyprogrammer.me/course_files/nlp/tmdb_5000_movies.csv

--2024-09-27 21:33:53--  https://lazyprogrammer.me/course_files/nlp/tmdb_5000_movies.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5698602 (5.4M) [text/csv]
Saving to: ‘tmdb_5000_movies.csv’

tmdb_5000_movies.cs 100%[===================>]   5.43M  --.-KB/s    in 0.09s   

2024-09-27 21:33:54 (62.5 MB/s) - ‘tmdb_5000_movies.csv’ saved [5698602/5698602]



In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [5]:
df = pd.read_csv('tmdb_5000_movies.csv')

In [6]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [7]:
x = df.iloc[0]
x

,0
budget,237000000
genres,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam..."
homepage,http://www.avatarmovie.com/
id,19995
keywords,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
original_language,en
original_title,Avatar
overview,"In the 22nd century, a paraplegic Marine is di..."
popularity,150.437577
production_companies,"[{""name"": ""Ingenious Film Partners"", ""id"": 289..."


# Read the First Row of the Pandas dataframe df
df is assumed to be a Pandas DataFrame.

    df.iloc[0]

is using the iloc function to select the first row from the DataFrame.

-------------------
## Explanation
iloc is an index-based selection method in Pandas, where you can select rows and columns by their integer positions (like you would in a NumPy array).

##df.iloc[0]

selects the first row in the DataFrame (since Python uses zero-based indexing).

This row is returned as a Pandas Series object, where the column names become the index, and the row's values become the data.

------------------------
#What happens with x?
x = df.iloc[0] stores the first row (a Series) of the DataFrame df into the variable x.

When you print or evaluate x, it will display the contents of the first row as a Series.

----------------------
#Example
If df looks like this:

    Name	Age	Gender
    Alice	30	  F
    Bob	  25  	M
    Carol	22	  F

Then, df.iloc[0] will output:

    Name      Alice
    Age          30
    Gender        F

Name: 0, dtype: object

So, x will hold this Series object, representing the first row of the DataFrame.

In [8]:
x['genres']

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [9]:
x['keywords']

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

# Extracting the Keys of a particular column
Each column in a Pandas DataFrame represents a distinct field, and each field can contain different types of data.

In this case, the keywords field contains JSON-like data (strings representing lists of dictionaries).

When you access x['keywords'], you're retrieving the value from just that column (the keywords column) for the specific row represented by x.

So, if x is the first row of the DataFrame, x['keywords'] will give you the JSON-like data stored in the keywords column for that row, not for other JSON fields like genres, production_companies, etc.

------------------------------
#Example
Assume x['keywords'] returns the following:

    '[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}]'

This is the content of the keywords column for this row only.

To clarify:

    x['genres']
    
retrieves data from the genres column for that row.

    x['keywords']
    
retrieves data from the keywords column for that row.

Other JSON columns like production_companies, production_countries, etc., will have their own specific content.

If you want to work with all JSON fields in the row, you'd have to access each one separately:

    x['genres']
    x['keywords']
    x['production_companies']
    x['production_countries']

Each of these will return the JSON content for that particular column in the specific row.

---------------------------
---------------------------
#Note:
The result you're getting from x['keywords'] is a list of dictionaries where each dictionary contains key-value pairs.

This means that the keywords column doesn't just store keys (as you might have expected), but rather JSON objects that include both an id and a name.

-----------------------
##Understanding the Output:

This result :

    [
    {"id": 1463, "name": "culture clash"},
    {"id": 2964, "name": "future"},
    {"id": 3386, "name": "space war"},
    {"id": 3388, "name": "space colony"},
    {"id": 3679, "name": "society"},
    {"id": 3801, "name": "space travel"},
    {"id": 9685, "name": "futuristic"},
    {"id": 9840, "name": "romance"},
    {"id": 9882, "name": "space"},
    {"id": 9951, "name": "alien"},
    {"id": 10148, "name": "tribe"},
    {"id": 10158, "name": "alien planet"},
    {"id": 10987, "name": "cgi"},
    {"id": 11399, "name": "marine"},
    {"id": 13065, "name": "soldier"},
    {"id": 14643, "name": "battle"},
    {"id": 14720, "name": "love affair"},
    {"id": 165431, "name": "anti war"},
    {"id": 193554, "name": "power relations"},
    {"id": 206690, "name": "mind and soul"},
    {"id": 209714, "name": "3d"}
    ]

This is a list of dictionaries where each dictionary has two key-value pairs:

##id: The unique identifier for the keyword.
##name: The actual keyword associated with the id.

---------------------------------
#Why is this happening?
In many movie databases (such as TMDB, where this data might come from), keywords are stored as structured JSON objects, where each keyword has both an id (for internal reference) and a name (the human-readable keyword).

When you access x['keywords'], you are retrieving the value stored in the keywords column for that specific row.

That value happens to be a JSON string representing a list of keyword objects, each containing both an id and a name.

----------------------
#Converting the JSON String to Python Objects:
To work with this JSON string in Python, you need to convert it from a string to a Python object (a list of dictionaries).

You can do this using the json.loads() function as shown earlier:
    import json

## Retrieve the 'keywords' column for the first row
    keywords_json = x['keywords']

## Parse the JSON string to a Python list of dictionaries
    keywords_list = json.loads(keywords_json)

## Now you can work with 'keywords_list' as a Python object
    print(keywords_list)

##Accessing Specific Parts:
Once you've converted the string to a list of dictionaries, you can loop through the list or access specific parts, such as the name of each keyword:

# Extract just the keyword names
    keyword_names = [keyword['name'] for keyword in keywords_list]

# Print the keyword names
    print(keyword_names)

This will output:


     ['culture clash', 'future', 'space war', 'space colony', 'society', 'space travel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alien planet', 'cgi', 'marine', 'soldier', 'battle', 'love affair', 'anti war', 'power relations', 'mind and soul', '3d']

--------------------
#Conclusion
The keywords column is not just a simple list of strings or keys, but a structured list of key-value pairs (id and name).

To handle this, you need to parse the JSON data and work with it in Python as a list of dictionaries.

In [10]:
j = json.loads(x['genres'])
j

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

# Load the JSON string in the genres column into a Python object
When you execute:

    j = json.loads(x['genres'])
    j
You're loading the JSON string in the genres column into a Python object.

The json.loads() function converts the JSON string into a Python list of dictionaries.

-----------------------------
-------------------------
#Example
###Assume the genres column contains this JSON-like string:

    '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}]'
---------------------
###After running:
    import json

    j = json.loads(x['genres'])
    print(j)

-----------------------------
###The result (j) will be a Python list of dictionaries:

    [
    {"id": 28, "name": "Action"},
    {"id": 12, "name": "Adventure"}
    ]

--------------------------------------
##The phrase:

"The json.loads() function converts the JSON string into a Python list of dictionaries"

##means the following:

###JSON string:
A JSON (JavaScript Object Notation) string is a text format for representing structured data, which is often used for transmitting data between a server and a web application.

It's similar to Python's dictionary and list structures. A JSON string looks like this:

    '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}]'

##Notice that this is a string (in quotes), but it contains structured data.

##json.loads():

This is a Python function from the json module, and it stands for "load string".

It takes a JSON-formatted string as input and converts it into a corresponding Python object.

##List of dictionaries:

In Python, a list is an ordered collection of items, and a dictionary is a collection of key-value pairs.

When a JSON string contains an array (like [...]), it is converted into a Python list. When it contains objects (like {"key": "value"}), it is converted into a Python dictionary.

---------------------
##In Context:
If you have a JSON string like:

   '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}]'

##It represents an array (list) of two objects (dictionaries), where each object has id and name fields.

When you use json.loads() on this string:

    import json
    json_string = '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}]'
    python_obj = json.loads(json_string)
    print(python_obj)

##The result (python_obj) will be a Python list of dictionaries:

    [
    {"id": 28, "name": "Action"},
    {"id": 12, "name": "Adventure"}
    ]

##Breakdown:
###JSON string: '[...]' → A text string formatted as JSON.
###Python list: [ ... ] → A list in Python, containing ordered items.
###Python dictionaries: { ... } → A dictionary in Python, where each item has a key-value pair.

In short, json.loads() transforms the textual JSON structure into native Python data structures (like lists and dictionaries), which you can then manipulate in your Python code.

Now, you can interact with j like any Python list. For example, to access the first genre:
  
    j[0]
    
This will give you:

    {'id': 28, 'name': 'Action'}

To access just the genre names:

    genre_names = [genre['name'] for genre in j]
    print(genre_names)

This will output:

    ['Action', 'Adventure']

---------------------
#Conclusion:
The json.loads() function converts the JSON string in x['genres'] into a usable Python object, which is a list of dictionaries in this case.

You can then easily work with this data, accessing individual elements or extracting specific values.

In [11]:
' '.join(''.join(jj['name'].split()) for jj in j)

'Action Adventure Fantasy ScienceFiction'

# Transform and Concatenate the "name" field from each dictionary in j

The expression:

    ' '.join(''.join(jj['name'].split()) for jj in j)
is doing several operations to transform and concatenate the "name" field from each dictionary in j, which is a list of dictionaries (like the result of the json.loads() operation).

---------------------------
-----------------------------
##Step-by-Step Breakdown:
    for jj in j

This is a loop that iterates through each dictionary in the list j.

Example of j:

    [
      {"id": 28, "name": "Action"},
      {"id": 12, "name": "Adventure"}
    ]

-----------------------------
Here, each jj will be a dictionary, e.g., {"id": 28, "name": "Action"}.

    jj['name']

This extracts the value of the "name" field from each dictionary.

For example:

For {"id": 28, "name": "Action"}, it will get "Action".

For {"id": 12, "name": "Adventure"}, it will get "Adventure".

------------------------------
    jj['name'].split()

The split() function splits the string based on whitespace (by default), turning it into a list of words.

If the name doesn't contain spaces (e.g., "Action"), split() will return a list with one element: ["Action"].
  
If name had multiple words (like "Science Fiction"), split() would return ["Science", "Fiction"].

-----------------------------------------
    ''.join(jj['name'].split())

This removes any whitespace between the words by rejoining the split list of words without spaces. For instance:

"Science Fiction" becomes ["Science", "Fiction"] after split(), and then ''.join() will return "ScienceFiction" (without a space).

---------------------------------
    ' '.join(...):

Finally, this joins all the processed "name" fields (now without spaces) with a single space ' '.

It concatenates all the modified "name" values together into one string.

###Example Walkthrough:
Suppose j contains:

    [
      {"id": 28, "name": "Action"},
      {"id": 12, "name": "Adventure"},
      {"id": 35, "name": "Science Fiction"}
    ]
The process works as follows:

For
    {"id": 28, "name": "Action"}, jj['name'].split() → ["Action"], ''.join(["Action"])
    
###→ "Action".

For

    {"id": 12, "name": "Adventure"}, jj['name'].split() → ["Adventure"], ''.join(["Adventure"])
    
###→ "Adventure".

For

    {"id": 35, "name": "Science Fiction"}, jj['name'].split() → ["Science", "Fiction"],

    ''.join(["Science", "Fiction"])
    
###→ "ScienceFiction".

Finally, ' '.join([...]) produces:

    "Action Adventure ScienceFiction"

--------------------
##Purpose
This code removes spaces within each "name" value and concatenates all the "name" fields from the list of dictionaries into a single string, with one space separating each modified name.

###' '.join(''.join(jj['name'].split()) for jj in j)
gives you the set of all values (the "universe of values") for the "name" field from all dictionaries within a particular field (such as genres or keywords) for a specific row in a DataFrame.

##Key points:
###"Universe of values":

It refers to all the values associated with the "name" field in the JSON-like data for that specific row.

This operation collects all those values and concatenates them into a single string.

###Particular field:

The code applies to a field (a column in the DataFrame) that stores JSON-like data, such as genres, keywords, production_companies, etc.

###Particular row:

This operation is performed on one specific row at a time, i.e., you're extracting and processing all the "name" values for that field in that row.

###Example:
Let’s assume you have a DataFrame df where the genres column in a particular row looks like this:

    '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Science Fiction"}, {"id": 16, "name": "Animation"}]'

If you extract this field for a specific row (using something like x = df.iloc[0]), and parse it with json.loads(x['genres']), you get:

    j = [
        {"id": 28, "name": "Action"},
        {"id": 12, "name": "Science Fiction"},
        {"id": 16, "name": "Animation"}
    ]

###Running:

    ' '.join(''.join(jj['name'].split()) for jj in j)
will give you:

    "Action ScienceFiction Animation"

This is the universe of "name" values from the genres field in that specific row, concatenated into a single string with spaces between them.

----------------------------
##Summary
The code extracts all the "name" values from the JSON-like field in a specific row of the DataFrame.

It joins those values (removing any internal spaces within names) and produces a single string.

This gives you the set of all values from that field in the row, which you're calling the "universe of values."

In [12]:
# convert the relevant data for each movie into a single string
# to be ingested by TfidfVectorizer
def genres_and_keywords_to_string(row):
  genres = json.loads(row['genres'])
  genres = ' '.join(''.join(j['name'].split()) for j in genres)

  keywords = json.loads(row['keywords'])
  keywords = ' '.join(''.join(j['name'].split()) for j in keywords)
  return "%s %s" % (genres, keywords)

# This function is designed to transform the genres and keywords fields from JSON-like formats into a text format that can be ingested by the TF-IDF Vectorizer in a machine learning pipeline

This code defines a function called genres_and_keywords_to_string that takes a row of a DataFrame as input and converts the genres and keywords columns into a single string.

This string is structured so that it can be ingested by a TF-IDF Vectorizer, commonly used in Natural Language Processing (NLP) tasks to convert text into numerical features.

--------------
#1. Function Definition:

    def genres_and_keywords_to_string(row):

The function takes a single row as input, where row is a row from a Pandas DataFrame.

Each row is expected to have at least two columns: 'genres' and 'keywords', which are JSON-like strings representing lists of dictionaries.

--------------------
#2. Extracting and Processing Genres:

    genres = json.loads(row['genres'])

The genres column contains a JSON string for each movie.
json.loads(row['genres']) parses this string and converts it into a Python list of dictionaries. For example, if row['genres'] is:

   '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}]'

It converts this into:

    [{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}]

--------------------------
#3. Joining the Genre Names:

    genres = ' '.join(''.join(j['name'].split()) for j in genres)

Purpose: To extract and concatenate the "name" field from each genre and remove any spaces within multi-word names.

-----------------------
##Step-by-step:

j['name'].split() splits the genre's name into a list of words.

''.join(j['name'].split()) removes any spaces within multi-word names (e.g., "Science Fiction" becomes "ScienceFiction").

' '.join(...) joins all the genre names into a single string, separated by a space.

-------------------------
##Example:

If the genre names are ["Action", "Adventure"], the result would be:

    "Action Adventure"
For a genre like "Science Fiction", the result would be:

    "ScienceFiction"

--------------------
#4. Extracting and Processing Keywords:
The same process is applied to the keywords column:

    keywords = json.loads(row['keywords'])
    keywords = ' '.join(''.join(j['name'].split()) for j in keywords)
json.loads(row['keywords']) converts the JSON string of keywords into a list of dictionaries.

It extracts the "name" field from each keyword, removes internal spaces, and concatenates them into a single string.

##Example: For keywords like:

    '[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}]'

The result would be:

    "cultureclash future"

----------------------
#5. Combining Genres and Keywords:

    return "%s %s" % (genres, keywords)

This returns a single string that combines the processed genres and keywords into one space-separated string.

###%s is a string formatting placeholder.

Example: If the genres string is "Action Adventure" and the keywords string is "cultureclash future", the result would be:

    "Action Adventure cultureclash future"

----------------------
##Purpose
This function is designed to transform the genres and keywords fields from JSON-like formats into a text format that can be ingested by the TF-IDF Vectorizer in a machine learning pipeline.

The Vectorizer will treat the processed string as a document, where words (genres and keywords) represent features, allowing the model to understand and learn from this information.

--------------------------
##TF-IDF Vectorizer:
TF-IDF stands for Term Frequency-Inverse Document Frequency. It is a technique to convert a collection of text documents into a numerical representation based on word frequency.

The output of this function will be a single string that can be vectorized using TfidfVectorizer for further processing, such as in a movie recommendation system.

In [13]:
# create a new string representation of each movie
df['string'] = df.apply(genres_and_keywords_to_string, axis=1)

# Create a new string column
This code creates a new column named 'string' in the DataFrame df.

This new column will contain a string representation of each movie, combining the genres and keywords information.

-------------------------

Here's a breakdown:

##df['string'] = ...:

This part assigns the result of the expression on the right side to a new column called 'string' in the DataFrame df.

##df.apply(genres_and_keywords_to_string, axis=1):

This applies the function genres_and_keywords_to_string to each row of the DataFrame.

##genres_and_keywords_to_string:

This is the function you defined earlier, which takes a row as input and returns a string combining the movie's genres and keywords.

##axis=1:

This specifies that the function should be applied row-wise (along each row).

In essence, this line of code iterates through each row of the DataFrame df, calls the genres_and_keywords_to_string function on each row, and stores the resulting string in the new 'string' column for that row.

In [14]:
# create a tf-idf vectorizer object
tfidf = TfidfVectorizer(max_features=2000)

# TF-IDF stands for Term Frequency-Inverse Document Frequency.

It's a numerical statistic that reflects how important a word is to a document in a collection or corpus.

-----------------------------
## TfidfVectorizer:

This is a class from the scikit-learn library in Python.

It's used to convert a collection of raw documents into a matrix of TF-IDF features.

----------------------
##tfidf = TfidfVectorizer(...):

This line creates an instance of the TfidfVectorizer class and assigns it to the variable tfidf.

This object will be used to perform the TF-IDF transformation on your text data.

-------------------------
##max_features=2000:

This is a parameter passed to the TfidfVectorizer constructor.

It specifies that you only want to consider the top 2000 most frequent words in your dataset when creating the TF-IDF matrix.

This can help reduce the dimensionality of your data and potentially improve performance.

In essence, this line of code sets up the tool you'll use to convert your text data into a numerical representation suitable for machine learning algorithms.

The tfidf object now holds the configuration and methods needed to perform the TF-IDF transformation.

-----------------------------
------------------------------
Here's how you would typically use it in a broader context:

    from sklearn.feature_extraction.text import TfidfVectorizer

----------------------
# Sample data
    documents = [
       "This is the first document.",
       "This document is the second document.",
       "And this is the third one.",
       "Is this the first document?"
      ]

--------------------
# Create a TfidfVectorizer object
    tfidf = TfidfVectorizer(max_features=2000)

---------------------
# Fit the vectorizer to the documents and transform them into a TF-IDF matrix
    tfidf_matrix = tfidf.fit_transform(documents)

------------------------
# You can now use tfidf_matrix for various machine learning tasks

#Important:

Make sure you have scikit-learn installed (!pip install scikit-learn) and import the TfidfVectorizer class before using it.

In [15]:
# create a data matrix from the overviews
X = tfidf.fit_transform(df['string'])

In [16]:
X

<4803x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 37285 stored elements in Compressed Sparse Row format>

In [17]:
# generate a mapping from movie title -> index (in df)
movie2idx = pd.Series(df.index, index=df['title'])
movie2idx

,0
title,
Avatar,0
Pirates of the Caribbean: At World's End,1
Spectre,2
The Dark Knight Rises,3
John Carter,4
...,...
El Mariachi,4798
Newlyweds,4799
"Signed, Sealed, Delivered",4800
